# kitti to nusc

In [2]:
import cv2
import numpy as np
from easydict import EasyDict

root = "dataset\\kitti\\"
raw  = root + "object\\training\\"
img2 = raw + "image_2\\"
img3 = raw + "image_3\\"
lidar =raw + "velodyne\\"
calib =raw + "calib\\"

sample_id  = 0
image_name = f"{sample_id:06d}.png"
lidar_name = f"{sample_id:06d}.bin"
calib_name = f"{sample_id:06d}.txt"

boundary = EasyDict({
    "minX": 0,
    "maxX": 50,
    "minY": -25,
    "maxY": 25,
    "minZ": -2.73,
    "maxZ": 1.27
})
BEV_WIDTH = 608   # across y axis -25m ~ 25m
BEV_HEIGHT = 608  # across x axis 0m ~ 50m
DISCRETIZATION_X = (boundary.maxX - boundary.minX) / BEV_HEIGHT
DISCRETIZATION_Y = (boundary.maxY - boundary.minY) / BEV_HEIGHT

In [5]:
import yaml
from pyquaternion import Quaternion

# read calib data
with open(calib+calib_name, 'r') as cfile:
    calib_dict = yaml.load(cfile, Loader=yaml.FullLoader)
calib_dict = {key: [float(v) for v in value.split(' ')]
              for key, value in calib_dict.items()}
# calib_dict

# Tr_velo_to_cam in KITTI format
kitti_to_nu_lidar = Quaternion(axis=(0, 0, 1), angle=np.pi / 2)
velo_to_cam_kitti = np.array(calib_dict['Tr_velo_to_cam']).reshape(3, 4)
print(f"{velo_to_cam_kitti=}")

rotat_expect = velo_to_cam_kitti[:3,:3]
trans_expect = velo_to_cam_kitti[:3, 3]

velo_to_cam_rot = velo_to_cam_kitti[:3, :3]
velo_to_cam_trans = velo_to_cam_kitti[:3, 3]
Tr_velo_to_cam = np.hstack((velo_to_cam_rot, velo_to_cam_trans.reshape(3, 1)))
print(f"{Tr_velo_to_cam=}")

raise Exception()

# DEBUG
velo_to_cam_kitti = np.array([[0.01122026, -0.99989861, -0.00876743, -0.00702234],
                              [0.05464516,  0.00936803, -0.99846189, -0.35150598],
                              [0.99844279,  0.0107239,  0.05474473, -0.7332409]])

# Inverse transformation matrices
velo_to_cam = np.dot(
    kitti_to_nu_lidar.transformation_matrix, velo_to_cam_kitti.transpose()).transpose()
# print(velo_to_cam)

# DEBUG
np.testing.assert_array_almost_equal(velo_to_cam, np.array(
    [[0.99989861,  0.01122026, -0.00876743, -0.00702234],
     [-0.00936803,  0.05464516, -0.99846189, -0.35150598],
     [-0.0107239,  0.99844279,  0.05474473, -0.7332409]]))
# velo_to_cam_org = np.array(
#     [[0.99989861,  0.01122026, -0.00876743, -0.00702234],
#      [-0.00936803,  0.05464516, -0.99846189, -0.35150598],
#      [-0.0107239,  0.99844279,  0.05474473, -0.7332409],
#      [0.,  0.,  0.,  1.]])

# TODO how can get lid_to_ego
# lid_to_ego = Quaternion(axis=(0, 0, -1), angle=np.pi / 2).transformation_matrix
# print(lid_to_ego)
# [[ 0. -1.  0.  0.]
#  [ 1.  0.  0.  0.]
#  [ 0.  0.  1.  0.]
#  [ 0.  0.  0.  1.]]
# [[-0.000542795004, 0.998930699, 0.0462294677, 0.985793],
# [-0.999995492, -0.000405693167, -0.00297501151, 0.0],
# [-0.00295307535, -0.0462308742, 0.998926417, 1.84019],
# [0.0, 0.0, 0.0, 1.0]]
# lid_to_ego = np.array([[0,  1,  0.046,  1],
#                        [-1,  0,  -0.003,  0],
#                        [-0.003,  -0.046,  1,  2],
#                        [0,  0,  0,  1]])

lid_to_ego = np.array(
    [[-5.42795004e-04,  9.98930699e-01,  4.62294677e-02, 9.85793000e-01],
     [-9.99995492e-01, -4.05693167e-04, -2.97501151e-03, 0.00000000e+00],
     [-2.95307535e-03, -4.62308742e-02,  9.98926417e-01, 1.84019000e+00],
     [0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])
# print(lid_to_ego)
np.set_printoptions(formatter={'float_kind': "{:.5f}".format}) # np format
ego_to_cam = np.dot(velo_to_cam_org, np.linalg.inv(lid_to_ego))
print(ego_to_cam)

rot_inv = ego_to_cam[:3,:3]
tr_temp = ego_to_cam[:3, 3]
trans   = np.linalg.inv(rot_inv) @ tr_temp
trans   = -trans.T
rotation= -rot_inv.T
print(f"{rotation=}")
print(f"{trans=}")

# cs_record_cam['rotation'] = Quaternion(matrix=ego_to_cam[:3, :3])
# cs_record_cam['translation'] = ego_to_cam[:3, 3]

velo_to_cam_kitti=array([[ 0.00692796, -0.9999722 , -0.00275783, -0.02457729],
       [-0.00116298,  0.00274984, -0.9999955 , -0.06127237],
       [ 0.9999753 ,  0.00693114, -0.0011439 , -0.3321029 ]])
Tr_velo_to_cam=array([[ 0.00692796, -0.9999722 , -0.00275783, -0.02457729],
       [-0.00116298,  0.00274984, -0.9999955 , -0.06127237],
       [ 0.9999753 ,  0.00693114, -0.0011439 , -0.3321029 ]])


Exception: 